# Interactive Time Series Visualization Assignment

In [7]:
!pip install plotly_express

In [8]:
import pandas as pd
import plotly_express as px
from ipywidgets import interact

### Load the Online Retail.xlsx Excel file into a Pandas dataframe.

In [9]:
df = pd.read_excel("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Time%20Series%20Analysis/Time%20Series%20-%20Day%202/Online%20Retail.xlsx")

In [10]:
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [12]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396034 entries, 0 to 396033
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396034 non-null  int64         
 1   InvoiceDate  396034 non-null  datetime64[ns]
 2   StockCode    396034 non-null  object        
 3   Description  396034 non-null  object        
 4   Quantity     396034 non-null  int64         
 5   UnitPrice    396034 non-null  float64       
 6   Revenue      396034 non-null  float64       
 7   CustomerID   396034 non-null  int64         
 8   Country      396034 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 27.2+ MB


In [14]:
df['Date'] = df['InvoiceDate'].dt.date
df['Month'] = df['InvoiceDate'].dt.strftime('%B')
df['Day'] = df['InvoiceDate'].dt.day
df['Weekday'] = df['InvoiceDate'].dt.strftime('%w')
df['Hour'] = df['InvoiceDate'].dt.hour
df['Minute'] = df['InvoiceDate'].dt.minute

df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Date,Month,Day,Weekday,Hour,Minute
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,1,3,8,26
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,1,3,9,2
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,1,3,9,32
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010-12-01,December,1,3,10,19
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010-12-01,December,1,3,10,39


### Create an interactive histogram to explore the distributions of daily (by date) Quantity, UnitPrice, and Revenue. Include a dropdown widget where you can switch between the three fields.

**Hint:** You will need to create a Date field and then aggregate on it. The Quantity and Revenue fields should be summed and the UnitPrice field should be averaged (mean).

In [43]:
mean_df = df.groupby("Date").mean()
mean_df = mean_df[["UnitPrice"]].reset_index()
mean_df.head()

,Date,UnitPrice
0,2010-12-01,3.109995
1,2010-12-02,3.184610
2,2010-12-03,3.166670
3,2010-12-05,2.840382
4,2010-12-06,2.878689


In [38]:
sum_df = df.groupby('Date').sum()
sum_df = sum_df[["Quantity", "Revenue"]].reset_index()
sum_df.head()

,Date,Quantity,Revenue
0,2010-12-01,24207,46219.29
1,2010-12-02,31140,47283.53
2,2010-12-03,11822,23576.01
3,2010-12-05,16370,31315.64
4,2010-12-06,16267,31014.21


In [56]:
sel = ["Quantity", "UnitPrice", "Revenue"]

@interact(Selection=sel)

def viz(Selection):
    print(f"You have selected: {Selection}")
    if Selection != "UnitPrice":
        fig = px.histogram(sum_df, x=Selection, title=Selection + " Distribution", nbins=50, template="none")
        fig.update_yaxes(title_text="Number of " + Selection)
        fig.update_traces(marker_color="lightskyblue",
                        marker_line_color="black",
                        marker_line_width=1)
        fig.show()
    else:
        fig = px.histogram(mean_df, x=Selection, title=Selection + " Distribution", nbins=50, template="none")
        fig.update_yaxes(title_text="Avg. " + Selection)
        fig.update_traces(marker_color="lightskyblue",
                        marker_line_color="black",
                        marker_line_width=1)
        fig.show()

interactive(children=(Dropdown(description='Selection', options=('Quantity', 'UnitPrice', 'Revenue'), value='Q…

### Create an interactive line chart where you can explore Quantity and Revenue by day (date) for each country. Include two dropdown widgets - one to choose between the Quantity and Revenue fields and the other to choose the Country.

In [60]:
ilc_df = df.groupby(["Date", "Country"]).sum()
ilc_df = ilc_df[["Quantity", "Revenue"]].reset_index()
ilc_df.head()

,Date,Country,Quantity,Revenue
0,2010-12-01,Australia,107,358.25
1,2010-12-01,EIRE,242,505.38
2,2010-12-01,France,446,801.86
3,2010-12-01,Germany,156,243.48
4,2010-12-01,Netherlands,96,177.60


In [61]:
dropbox = ["Quantity", "Revenue"]
country = list(df["Country"].unique())

@interact(dropbox=sorted(dropbox), country=sorted(country))

def viz(dropbox, country):
    temp_df = ilc_df[ilc_df["Country"] == country]
    fig = px.line(temp_df, x="Date", y=dropbox, title="{} v. Date for {}".format(dropbox, country))
    fig.show()

interactive(children=(Dropdown(description='dropbox', options=('Quantity', 'Revenue'), value='Quantity'), Drop…

### Create an interactive scatter plot showing the relationships between daily Quantity, UnitPrice, and Revenue for the United Kingdom. Include two dropdown boxes that let you choose between the 3 fields - one for the x axis and one for the y axis of your scatter plot. Size the data points according to Revenue.

In [63]:
isp_df = pd.concat([sum_df, mean_df["UnitPrice"]], axis=1)
isp_df.head()

,Date,Quantity,Revenue,UnitPrice
0,2010-12-01,24207,46219.29,3.109995
1,2010-12-02,31140,47283.53,3.184610
2,2010-12-03,11822,23576.01,3.166670
3,2010-12-05,16370,31315.64,2.840382
4,2010-12-06,16267,31014.21,2.878689


In [64]:
dd = ["Quantity", "Revenue", "UnitPrice"]

@interact(d1=sorted(dd), d2=sorted(dd))

def viz(d1, d2):
    fig = px.scatter(isp_df, x=d1, y=d2, size="Revenue", title="Interactive Scatter Plot")
    fig.update_traces(marker_line_color="black", marker_line_width=0.1)
    fig.show()

interactive(children=(Dropdown(description='d1', options=('Quantity', 'Revenue', 'UnitPrice'), value='Quantity…

### Create a bar chart showing the top X products sold (by quantity) in the United Kingdom in a specific month. Use a dropdown box to select the month name and a slider to show the top X products. The range for X should be from 5 to 25. Make sure the bars are sorted in descending order according to their total quantity sold for the month.

Bonus points if you can figure out how to ensure that the month names in the dropdown appear in their correct order (January, February, March, April, etc.).

In [66]:
bc_df = df[df.Country == "United Kingdom"]
bc_df = bc_df.groupby(["Month", "StockCode"]).sum()["Quantity"].reset_index().sort_values("Quantity", ascending=False)
bc_df["StockCode"] = bc_df["StockCode"].astype(str)
bc_df.head()

,Month,StockCode,Quantity
1615,April,84077,9552
25353,October,84077,8078
21390,November,22197,8036
5407,December,22197,7012
19194,May,22197,6618


In [70]:
months = list(bc_df.Month.unique())

In [75]:
@interact(month=sorted(months), top=(5,25))

def viz(month, top):
    temp_df = bc_df[bc_df["Month"] == month]
    temp_df = temp_df.head(top)
    fig = px.bar(temp_df, "StockCode", "Quantity")
    fig.show()

interactive(children=(Dropdown(description='month', options=('April', 'August', 'December', 'February', 'Janua…